In [2]:
import sys

# Need to find a better way of doing this.
sys.path.append("..")

import core.groups as groups
import core.node as node
import device.polynomial3K_camera as poly3K_camera
import torch

fake_SubsystemModel = node.Node(name="fake ET subsystem")
transform_toSubsystemModel_fromCamera = groups.SE3.create_identity()
point_in3D = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

camera = \
    poly3K_camera.Polynomial3KCamera(
        fake_SubsystemModel,
        transform_toSubsystemModel_fromCamera,
        requires_grad=True
    )

point_inPixels = camera.project_toPixels_fromCamera(point_in3D)
point_inImagePlane = camera.get_point_inImagePlane(point_inPixels)

distorted_point_inImagePlane = \
    camera.distort_point_inUndistortedImage(point_inImagePlane)

# New method is 3x faster than old (alt_) method.
%timeit camera.undistort_point_inDistortedImage(distorted_point_inImagePlane)
%timeit camera.alt_undistort_point_inDistortedImage(distorted_point_inImagePlane)

112 µs ± 1.64 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
341 µs ± 5.98 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
